In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,IsolationForest
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from datetime import datetime
from sklearn.preprocessing import StandardScaler,MinMaxScaler
!pip install --quiet google_trans_new
from google_trans_new import google_translator
from sklearn.impute import KNNImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sales_train=pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
item_categories=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
items=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
shops=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv', index_col='ID')

In [ ]:
sales_train.info()

In [ ]:
sales_train['date']=pd.to_datetime(sales_train['date'],dayfirst=True)

In [ ]:
sales_train.groupby(['date_block_num'])['item_cnt_day'].sum().plot(kind='bar',figsize=(20,6))

# Feature engineering

In [ ]:
sales_train.columns, items.columns, shops.columns, test.columns,item_categories.columns

In [ ]:
sales_train.describe().T

In [ ]:
plt.rcParams['figure.figsize']=(20,8)
sns.boxplot(x=sales_train.item_price)

In [ ]:
sns.boxplot(x=sales_train.item_cnt_day)

In [ ]:
#Remove outliers
#sales_train=sales_train[sales_train.item_cnt_day<1001]
#sales_train=sales_train[sales_train.item_price<100000]

In [ ]:
sales_train.loc[sales_train.item_price<0, ['date_block_num','shop_id','item_id','item_price']]

In [ ]:
sales_train.loc[sales_train.item_price<0,'item_price']=sales_train[
    (sales_train.date_block_num==4)&(sales_train.shop_id==32)
    &(sales_train.item_id==2973)
    &(sales_train.item_price>0)].item_price.mean()

In [ ]:
#Drop duplicates from tain and test, all columns
sales_train=sales_train.drop_duplicates(keep = 'last').reset_index(drop = True)
sales_train=sales_train.drop_duplicates(subset=['date','date_block_num','shop_id','item_id','item_cnt_day'],keep = 'last').reset_index(drop = True)

In [ ]:
sales_train.describe().T

In [ ]:
#Drop duplicates from columns
print(sales_train[sales_train.duplicated(['date','date_block_num','shop_id','item_id'])])

In [ ]:
#Correction 1
sales_train.loc[(sales_train.date=='27.01.2013')&(sales_train.date_block_num==0)&(sales_train.shop_id==38)&(sales_train.item_id==15702),'item_price']=sales_train.loc[(sales_train.date=='27.01.2013')&(sales_train.date_block_num==0)&(sales_train.shop_id==38)&(sales_train.item_id==15702)].item_price.mean()
sales_train.loc[(sales_train.date=='27.01.2013')&(sales_train.date_block_num==0)&(sales_train.shop_id==38)&(sales_train.item_id==15702),'item_cnt_day']=sales_train.loc[(sales_train.date=='27.01.2013')&(sales_train.date_block_num==0)&(sales_train.shop_id==38)&(sales_train.item_id==15702)].item_cnt_day.sum()
#Correction 2
sales_train.loc[(sales_train.date=='29.12.2013')&(sales_train.date_block_num==11)&(sales_train.shop_id==27)&(sales_train.item_id==12133),'item_price']=sales_train.loc[(sales_train.date=='29.12.2013')&(sales_train.date_block_num==11)&(sales_train.shop_id==27)&(sales_train.item_id==12133)].item_price.mean()
sales_train.loc[(sales_train.date=='29.12.2013')&(sales_train.date_block_num==11)&(sales_train.shop_id==27)&(sales_train.item_id==12133),'item_cnt_day']=sales_train.loc[(sales_train.date=='29.12.2013')&(sales_train.date_block_num==11)&(sales_train.shop_id==27)&(sales_train.item_id==12133)].item_cnt_day.sum()
#Correction 3
sales_train.loc[(sales_train.date=='26.12.2014')&(sales_train.date_block_num==23)&(sales_train.shop_id==17)&(sales_train.item_id==3424),'item_price']=sales_train.loc[(sales_train.date=='26.12.2014')&(sales_train.date_block_num==23)&(sales_train.shop_id==17)&(sales_train.item_id==3424)].item_price.mean()
sales_train.loc[(sales_train.date=='26.12.2014')&(sales_train.date_block_num==23)&(sales_train.shop_id==17)&(sales_train.item_id==3424),'item_cnt_day']=sales_train.loc[(sales_train.date=='26.12.2014')&(sales_train.date_block_num==23)&(sales_train.shop_id==17)&(sales_train.item_id==3424)].item_cnt_day.sum()
#Correction 4
sales_train.loc[(sales_train.date=='17.02.2015')&(sales_train.date_block_num==25)&(sales_train.shop_id==5)&(sales_train.item_id==21619),'item_price']=sales_train.loc[(sales_train.date=='17.02.2015')&(sales_train.date_block_num==25)&(sales_train.shop_id==5)&(sales_train.item_id==21619)].item_price.mean()
sales_train.loc[(sales_train.date=='17.02.2015')&(sales_train.date_block_num==25)&(sales_train.shop_id==5)&(sales_train.item_id==21619),'item_cnt_day']=sales_train.loc[(sales_train.date=='17.02.2015')&(sales_train.date_block_num==25)&(sales_train.shop_id==5)&(sales_train.item_id==21619)].item_cnt_day.sum()
#Drop duplicates again
sales_train=sales_train.drop_duplicates(keep = 'last').reset_index(drop = True)

In [ ]:
iso=IsolationForest(contamination=0.005)
iso.fit(sales_train[['shop_id','item_id','item_price','item_cnt_day']])
sales_train['anomaly']=iso.predict(sales_train[['shop_id','item_id','item_price','item_cnt_day']])

# Add additional features

In [ ]:
#df=sales_train[sales_train.anomaly==1]
sales_train.loc[sales_train.anomaly==-1,['item_price']]=np.nan

In [ ]:
impt=KNNImputer(n_neighbors=3)
replace_nan=impt.fit_transform(sales_train.drop(['date','anomaly','item_cnt_day'],axis=1).values)

In [ ]:
df=sales_train.copy()
df['item_price']=replace_nan[:,-1]

In [ ]:
#Convert date to proper datetime and extract month and year
df['date']=pd.to_datetime(df.date,dayfirst=True)
df['month']=df.date.dt.month
df['year']=df.date.dt.year

In [ ]:
#create revenue column
df['revenue']=df.item_cnt_day*df.item_price

In [ ]:
#Translate shop name and extract city
translator=google_translator()
shops['shop_name_en']=shops.shop_name.apply(translator.translate)

In [ ]:
shops['city']=[x[0] for x in shops.shop_name_en.str.replace('!','').str.strip().str.replace(',','').str.split(' ')]

In [ ]:
shops.city.unique()

In [ ]:
shops.loc[shops.city=='SPb','city']='SPB'
shops.loc[shops.city=='1c-online','city']='Online'
shops.loc[shops.city=='Voronež','city']='Voronezh'

In [ ]:
shops['city_code']=shops.city.map(dict(zip(shops.city.unique(),range(len(shops.city.unique())))))

In [ ]:
#df=df.merge(shops,on=['shop_id'],how='left').drop(['shop_name','shop_name_en','city'],axis=1)

In [ ]:
df=df.groupby(['date_block_num','shop_id', 'item_id', 
        'month', 'year'],as_index=False)['item_price','item_cnt_day','revenue'].agg({'item_price': 'mean','item_cnt_day':'sum','revenue':'sum'})

**Merge test data and add some more feature**

In [ ]:
#test.drop(['ID'],axis=1,inplace=True)
test['year']=2015
test['month']=11
test['date_block_num']=34
test.head()

In [ ]:
df=df.merge(test,on=['shop_id', 'item_id', 'year', 'month', 'date_block_num'],how='outer').merge(items,on=['item_id'],how='left').merge(item_categories,on=['item_category_id'],how='left').drop(['item_name','item_category_name'],axis=1).merge(shops,on=['shop_id'],how='left').drop(['shop_name','shop_name_en','city'],axis=1)

In [ ]:
df.describe().T

In [ ]:
df['days']=df.month.map({1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31})

In [ ]:
'''tot_df=df.drop(['year','month'],axis=1)
for i in range(4):
    tot_df['price_lag_'+str(i)]=tot_df.groupby(['date_block_num','shop_id','item_id'])['item_price'].mean().shift(i).values
    tot_df['item_cnt_lag_'+str(i)]=tot_df.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].mean().shift(i).values
    tot_df['revenue_lag_'+str(i)]=tot_df.groupby(['date_block_num','shop_id','item_id'])['revenue'].mean().shift(i).values
'''

In [ ]:
tot_df=df.drop(['month','year','ID'],axis=1)

In [ ]:
tot_df_lag_1=tot_df.copy()
tot_df_lag_1['date_block_num']=tot_df_lag_1['date_block_num']+1
tot_df_lag_1=tot_df_lag_1[tot_df_lag_1['date_block_num']<35]
tot_df_lag_1.rename(columns={'item_cnt_day':'item_cnt_day_lag1'},inplace=True)
tot_df_lag_2=tot_df.copy()
tot_df_lag_2['date_block_num']=tot_df_lag_2['date_block_num']+2
tot_df_lag_2=tot_df_lag_2[tot_df_lag_2['date_block_num']<35]
tot_df_lag_2.rename(columns={'item_cnt_day':'item_cnt_day_lag2'},inplace=True)

In [ ]:
tot_df_lag_1.item_cnt_day_lag1.value_counts()

In [ ]:
tot_df=tot_df.merge(tot_df_lag_1,on=['date_block_num','shop_id','item_id','revenue','item_category_id','item_price','city_code','days'],how='left').merge(tot_df_lag_2,on=['date_block_num','shop_id','item_id','revenue','item_category_id','item_price','city_code','days'],how='left')

In [ ]:
tot_df.fillna(0,inplace=True)

In [ ]:
X_train=tot_df[tot_df.date_block_num<33].drop(['item_cnt_day'],axis=1)
y_train=tot_df[tot_df.date_block_num<33]['item_cnt_day']
X_valid=tot_df[(tot_df.date_block_num==33)].drop(['item_cnt_day'],axis=1)
y_valid=tot_df[(tot_df.date_block_num==33)]['item_cnt_day']
X_test=tot_df[tot_df.date_block_num==34].drop(['item_cnt_day'],axis=1)

In [ ]:
X_train.columns

In [ ]:
categorical_feature_names=['shop_id','item_id','item_category_id','city_code']
feature_names = X_train.columns.tolist()

params = {
    'objective': 'mse',
    'metric': 'rmse',
    'num_leaves': 255,
    'learning_rate': 0.005,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 5,
    'seed': 1,
    'verbose': 1,
    'force_row_wise' : True
}

In [ ]:
lgb_train = lgb.Dataset(X_train[feature_names], y_train, categorical_feature=None)
lgb_eval  = lgb.Dataset(X_valid[feature_names], y_valid, categorical_feature=None, reference=lgb_train)

In [ ]:
#rfr=RandomForestRegressor(n_estimators=200)
#rfr.fit(X_train,y_train)

In [ ]:
#y_pred=rfr.predict(X_valid)

In [ ]:
#mean_squared_error(y_valid,y_pred)

In [ ]:
evals_result = {}
gbm = lgb.train(
        params, 
        lgb_train,
        num_boost_round = 4000,
        valid_sets = (lgb_train, lgb_eval), 
        feature_name = feature_names,
        categorical_feature = categorical_feature_names,
        verbose_eval = 100, 
        evals_result = evals_result,
        early_stopping_rounds = 100)

In [ ]:
lgb.plot_importance(
    gbm, 
    max_num_features=50, 
    importance_type='gain', 
    figsize=(12,8));

In [ ]:
sample_submission['item_cnt_month'] = gbm.predict(X_test[feature_names]).clip(0, 20)
sample_submission.to_csv('submission_manj.csv')